In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import math
import sys
from src import functions as func
from src import feagen as feag
import datetime
import numpy as np
from datetime import datetime,timedelta
from keras.models import Sequential
from keras.layers import Input, Dense,GRU,Activation
from keras.layers.core import Flatten,Dropout
from sklearn import preprocessing
import keras
%matplotlib inline

task = 'am'
target = 'A-3'

if task == 'am':
    window_start = datetime.strptime('2016-10-18 08:00:00', "%Y-%m-%d %H:%M:%S")
elif task == 'pm':
    window_start = datetime.strptime('2016-10-18 17:00:00', "%Y-%m-%d %H:%M:%S")
else:
    sys.exit("Usage : Only am or pm for <task>")
    
windows = []
for d in range(7):
    for t in range(6):
        windows.append('\"['+str(window_start +timedelta(minutes=20*t))+','+ str(window_start +timedelta(minutes=20*(t+1)))+')\"')
    window_start  += timedelta(days=1)

def create_dataset(dataset,lookback = 1):
    dataX,dataY = [],[]
    for day in dataset:
        for i in range(len(day)-look_back):
            a = day[i:(i+look_back), 0]
            dataX.append(a)
            dataY.append([day[i + look_back, 0]])
    return np.array(dataX), np.array(dataY)

    
    

def my_mape(label,pred):
    c = 0.0
    mae = 0.0
    for (l,p) in zip(label.flatten(),pred.flatten()):
        if l != 0:
            mae += abs(l-p)/l
            c += 1
    return mae/c,c

Using TensorFlow backend.


# Load data

In [2]:

win_6 = (6-6)*3
win_8 = (8-6)*3
win_10 = (10-6)*3
win_15 = (15-6)*3
win_17 = (17-6)*3
win_19 = (19-6)*3

if task == 'am':
    x_range = range(win_6,win_8)
    y_range = range(win_8,win_10)
    xy_range = range(win_6,win_10)
else:
    x_range = range(win_15,win_17)
    y_range = range(win_17,win_19)
    xy_range = range(win_15,win_19)

#Read training data
path = '../dataSets/training/'
file_suffix = '.csv'
in_file = 'trajectories(table 5)_training'
travel_times = func.read_file_to_travel_times(path, in_file, file_suffix)


#Read testing data
path = '../dataSets/testing_phase1/'
file_suffix = '.csv'
in_file = 'trajectories(table 5)_test1'
testing_times = func.read_file_to_travel_times(path, in_file, file_suffix)

#merge them into one dict
routes =  travel_times.keys()
for r in routes:
    travel_times[r].update(testing_times[r])

total_weeks = 6 # ?(train) + 1(valid) + 1(test), take off 10/1~7
training_weeks = total_weeks-2
testing_weeks = total_weeks-1
num_instance = total_weeks*7 # 13 weeks
num_window = 13*3 # 6 ~ 19
routes = travel_times.keys()
print routes
window_size = timedelta(minutes=20)
window_start = datetime.strptime('2016-09-13 06:00:00', "%Y-%m-%d %H:%M:%S")
window_end = datetime.strptime('2016-09-13 19:00:00', "%Y-%m-%d %H:%M:%S")


cube_2d = np.zeros((num_instance*num_window))

d=0
while d < num_instance:
    window_scanner = window_start
    for t in range(num_window):
        if str(window_scanner.date())>='2016-10-18' and str(window_scanner.date())<='2016-10-24' and str(window_scanner.time())>='06:00:00' and str(window_scanner.time())<'10:00:00' :
            print str(window_scanner),
            try:
                print travel_times[target][window_scanner]
            except:
                print ' None'
        try:
            cube_2d[d*39+t] = np.mean(travel_times[target][window_scanner])
        except:
            cube_2d[d*39+t] = 0
        window_scanner += window_size
    window_start += timedelta(days=1)
    d += 1


['B-3', 'B-1', 'A-3', 'A-2', 'C-3', 'C-1']
2016-10-18 06:00:00 [78.4, 104.14, 25.9]
2016-10-18 06:20:00 [100.92, 183.1]
2016-10-18 06:40:00 [549.98, 169.37, 212.71, 172.41, 121.7, 175.91]
2016-10-18 07:00:00 [120.02, 116.48, 197.91, 140.18]
2016-10-18 07:20:00 [125.99000000000001, 143.54, 56.08, 118.43, 152.69, 153.87, 242.24, 219.23]
2016-10-18 07:40:00 [175.8, 221.99, 201.78, 217.81, 221.5, 180.24, 216.92000000000002, 225.64]
2016-10-18 08:00:00  None
2016-10-18 08:20:00  None
2016-10-18 08:40:00  None
2016-10-18 09:00:00  None
2016-10-18 09:20:00  None
2016-10-18 09:40:00  None
2016-10-19 06:00:00 [113.62, 65.4]
2016-10-19 06:20:00 [197.56, 187.63, 107.96000000000001, 153.39]
2016-10-19 06:40:00 [205.84, 81.53999999999999, 138.78, 107.57]
2016-10-19 07:00:00 [108.24, 102.68, 181.98, 177.0, 120.84, 148.57]
2016-10-19 07:20:00 [206.35, 138.17000000000002, 191.75, 168.46, 160.15, 165.73]
2016-10-19 07:40:00 [153.78, 228.87, 229.65, 120.56, 140.03, 276.43, 209.61, 297.69, 264.58, 191.56

# Feature Engineering

In [3]:
# Missing value inputation....
### to be done
# Normalize
cube_2d = cube_2d.reshape(-1, 1)
scaler = preprocessing.MinMaxScaler(feature_range=(0, 1))
normed = scaler.fit_transform(cube_2d)
#normed = preprocessing.normalize(cube_2d,axis=0)

cube_3d = np.zeros((num_instance,num_window,1))
for d in range(num_instance):
    for t in range(num_window):
        cube_3d[d][t][0] = normed[d*39+t]

train = cube_3d[:training_weeks*7,xy_range,:]
valid = cube_3d[training_weeks*7:testing_weeks*7,xy_range,:]

test = cube_3d[testing_weeks*7:,x_range,:]


look_back = 6

trX, trY = create_dataset(train, look_back)
valX,valY = create_dataset(valid,look_back)


trX = np.reshape(trX, (trX.shape[0], 1, trX.shape[1]))
valX = np.reshape(valX, (valX.shape[0], 1, valX.shape[1]))

print 'Train data:',trX.shape,trY.shape
print 'Valid data:',valX.shape,valY.shape
print 'Test data:',test.shape
print 'Valid non-zero',np.count_nonzero(valY.flatten())
print 'Test non-zero:',np.count_nonzero(test)



Train data: (168, 1, 6) (168, 1)
Valid data: (42, 1, 6) (42, 1)
Test data: (7, 6, 1)
Valid non-zero 42
Test non-zero: 42


In [4]:
print test

[[[ 0.13234244]
  [ 0.27049438]
  [ 0.44510335]
  [ 0.27361342]
  [ 0.28858718]
  [ 0.39563684]]

 [[ 0.1704947 ]
  [ 0.30787521]
  [ 0.25415634]
  [ 0.26644677]
  [ 0.32717674]
  [ 0.41171139]]

 [[ 0.21533265]
  [ 0.19365653]
  [ 0.3832396 ]
  [ 0.30635331]
  [ 0.27415967]
  [ 0.38683052]]

 [[ 0.21363742]
  [ 0.21446789]
  [ 0.26963248]
  [ 0.22663261]
  [ 0.36417571]
  [ 0.30718421]]

 [[ 0.19295177]
  [ 0.17868277]
  [ 0.22499452]
  [ 0.21440249]
  [ 0.27257056]
  [ 0.28101498]]

 [[ 0.1770915 ]
  [ 0.15538363]
  [ 0.22210882]
  [ 0.16261536]
  [ 0.15561855]
  [ 0.25638649]]

 [[ 0.20192698]
  [ 0.40030349]
  [ 0.26806582]
  [ 0.32913093]
  [ 0.34887746]
  [ 0.40672675]]]


In [5]:
def eval_model(valX,valY,scaler,model):
    predY = []
    for day in range(7):
        val = np.reshape(valX[day*6,:,:],(1,1,6))
        for win in range(6):
            pred = model.predict(val)
            val = np.append(val[0,0,1:],pred)
            val = np.reshape(val,(1,1,6))
            predY.append(pred)
    predY = np.array(predY).reshape(1, -1)
    predY = scaler.inverse_transform(predY)
    raw_valY = scaler.inverse_transform(valY)
    
    return my_mape(raw_valY,predY)

def record(model,scaler,mape,count,step,task,target,test,LSTM_out,training_weeks,windows):
    #record mape
    with open('./rnn_record/'+str(training_weeks)+'w_'+target+task+'.txt','w') as record:
        record.write('Mape :'+str(mape))
        record.write(' Count :'+str(count))
        record.write(' Epoch:'+str(step))
        record.write(' LSTM out :'+str(LSTM_out)+'\n')
    #make prediction file
    with open('./rnn_record/'+str(training_weeks)+'w_'+target+task+'.csv','w') as record:
        record.write('\"intersection_id\",\"tollgate_id\",\"time_window\",\"avg_travel_time\"\n')
        flag = 0
        for day in range(7):
            ttX = np.array(test[day,:,:])
            ttX = np.reshape(ttX,(1,1,6))
            for i in range(6):
                record.write(target.split('-')[0]+',')
                record.write(target.split('-')[1]+',')
                record.write(windows[flag]+',')
                flag +=1
                #predict
                pred = model.predict(ttX)
                ttX = np.append(ttX[0,0,1:],pred)
                ttX = np.reshape(ttX,(1,1,6))
                record.write(str(scaler.inverse_transform(pred).flatten()[0])+'\n')

# Model

In [7]:
epoch = 300
diplay = 1
step = epoch/diplay
LSTM_out = 20
best = 0.3

sample_w = np.zeros(trY.shape[0])
for i in range(len(trY)):
    sample_w[i] = 0 if trY[i,0] == 0 else 1/trY[i,0]
    

# create and fit the LSTM network
print "Doing route ",target,' LSTM_out ',LSTM_out
model = Sequential()
model.add(GRU(LSTM_out, input_shape=(1, look_back)))
model.add(Dense(1))
adam = keras.optimizers.Adam(lr=0.001, decay=1e-6)
model.compile(loss='mae', optimizer=adam)
loss= []
for i in range(step):
    hist = model.fit(trX, trY, epochs=diplay, verbose=0,sample_weight=sample_w)
    mape,count = eval_model(valX,valY,scaler,model)
    loss.append(hist.history['loss'])
    print 'Step ',(i+1)*diplay,' valid score : ',mape
    if best > mape:
        best = mape
        #record(model,scaler,mape,count,(i+1)*diplay,task,target,test,LSTM_out,training_weeks,windows)
print 'Best mape = ',best
print model.predict(valX)

    
    
    



Doing route  A-3  LSTM_out  20
Step  1  valid score :  0.980834253308
Step  2  valid score :  0.884705335692
Step  3  valid score :  0.77791949414
Step  4  valid score :  0.657707640773
Step  5  valid score :  0.522464939694
Step  6  valid score :  0.37176492699
Step  7  valid score :  0.27332145913
Step  8  valid score :  0.264587526981
Step  9  valid score :  0.265223611637
Step  10  valid score :  0.263592501521
Step  11  valid score :  0.26936195574
Step  12  valid score :  0.282620649289
Step  13  valid score :  0.295680807932
Step  14  valid score :  0.295075870987
Step  15  valid score :  0.291057841419
Step  16  valid score :  0.282986023932
Step  17  valid score :  0.282562517719
Step  18  valid score :  0.282351648565
Step  19  valid score :  0.280595512729
Step  20  valid score :  0.288709380063
Step  21  valid score :  0.292839731168
Step  22  valid score :  0.291822534106
Step  23  valid score :  0.280708247801
Step  24  valid score :  0.275458493988
Step  25  valid score 

KeyboardInterrupt: 

In [ ]:
plt.plot(loss)